# Clustering Neghborhoods in Toronto

Using the Toronto neighborhood classification from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, we will explore the neighborhood using the Foursquare API and then create clusters using k-Means Clustering ALgorithm. Finally, we will map the clusters using Folium  

First,we import all the necessary libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import json 
from geopy.geocoders import Nominatim
from pandas import json_normalize
from sklearn.cluster import KMeans
import folium 

print('Libraries imported.')

Libraries imported.


Next, we use BeautifulSoup4 to parse the HTML script of the link above and find the table containing the pertinent information, i.e. the neighborhoods and postal codes. 

In [2]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(url.content,'html.parser')
interest = soup.find('table', class_='wikitable sortable')
rows = interest.find('tbody').find_all('tr')
rows_array=[] #to convert into ndarray
for x in rows:
    x_lst = [] #this would be a single row
    for y in x.find_all('td'):
        x_lst.append(y.get_text())
    x_lst = list(map(lambda s: s.strip(), x_lst)) #stripping the rows of whitespace characters that were carried over from the HTML script
    rows_array.append(x_lst)

We then map `rows_array` to a dataframe, `df`.

In [13]:
df = pd.DataFrame(rows_array)
df.rename(columns={0:'Postal Code', 1:'Borough', 2:'Neighborhood'}, inplace=True)
df.drop([0], inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
1,M1A,Not assigned,
2,M2A,Not assigned,
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [16]:
df.reset_index(inplace=True)
df.drop(columns=['index'], inplace=True)
df.head(10)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"


Getting the dimensions of the dataframe

In [28]:
df.shape

(180, 3)

Removing rows where Borough isn't assigned

In [29]:
df = df[df['Borough']!='Not assigned']
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


The new dimensions are now:

In [31]:
df.shape

(103, 3)